In [6]:
%load_ext autoreload
%autoreload 2

from hyper_qa.model import HyperQA
import numpy as np
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
embedding_matrix = np.random.normal(0, 1, (1000, 300))
sample_sequences = []
lengths = []
def generate_samples(max_length, sample_size, vocab_size=1000, pad=False):
    for i in range(sample_size):
        text = np.random.randint(1,1000, np.random.randint(1,max_length))
        if pad:
            sample = np.concatenate((text, [0] * (max_length - len(text))))
        else:
            sample = text
        lengths.append(len(text))
        sample_sequences.append(sample)

    return samples

In [12]:
q = generate_samples(50, 200)
a = generate_samples(50, 200)
b = generate_samples(50, 200)

In [8]:
model = HyperQA(1000, max_length, embedding_matrix=embedding_matrix)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
logits = model()